In [14]:
import sys
import os
import time
import operator
import pandas as pd
import re

class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self):
        super().__init__()
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def __str__(self):
        return 'total count: {}, distinct element: {}'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return "'{}', count: {}".format(self.ID,self.count)

def find(e,element_list):
    try:
        index = [ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index  
    
    
start=time.time()


# path='X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs'
path=r'C:\Users\Pero\python\Python\webdocs'
pattern='out_.*'
r=re.compile(pattern)
fileli=list(filter(r.match,os.listdir(path)))

size=1536
Top=[]
#item_count=100000
for item in fileli[:1]:
    with open(os.path.join(path,item),'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print('EOF')
                break
            else:
                item=Tail(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if len(Top)==0:
                    Top.append(item)
                else:
                    index=find(item,Top)
                    if index<0:
                        if len(Top)<size:
                            Top.append(item)
                            index=len(Top)-1
                        else:
                            # replace last element with count 
                            Top[-1].ID=item.ID
                            Top[-1].count+=1
                    else:
                        Top[index].count+=1
                    if index==0 or Top[index].count< Top[index-1].count:
                        pass
                    else:
                        Top.sort(key=operator.attrgetter('count'),reverse=True)                      

end=time.time()
print(Top[:20],len(Top))
print("Top-{}".format(size))
print("Total memory {0} bytes=Top-{1} with size {0} bytes.".format(sys.getsizeof(Top),size))
print("Execution time:{:8.3f} seconds.".format(end-start))

#　conver Top into df    
templi=[[i.ID,i.count] for i in Top]
df=pd.DataFrame(templi,columns=['ID', 'Count'])

#path=r"X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs\\"
path='C:\\Users\\Pero\\python\\Python\\webdocs\\'
name="SS_webdocs_00"+'_'+str(size)
df.to_csv(path+name+".csv",index=False)

#====================result compare=============================
groundtruth='webdocs_00_ground_truth.csv'
final=name+".csv"

grtruth=pd.read_csv(os.path.join(path,groundtruth),nrows=size)
My_result=pd.read_csv(os.path.join(path,final))

# precision
gt_set=set(grtruth['Element'][:size])
    # Top-size of ground truth
my_set=set(My_result['ID'])
precision=len(gt_set & my_set)/len(my_set)
    # &: set 交集運算
        
# ARE/AAE
gt_dict=dict(grtruth.values.tolist())
my_dict=dict(My_result.values.tolist())
distinct=len(gt_dict)
are_error=0
aae_error=0
tp=0
fp=0

for item in gt_dict:
    # Overall ARE/AAE
    if item in my_dict:
        # ARE/AAE for Top
        are_error+=abs(my_dict[item]-gt_dict[item])/my_dict[item]
        aae_error+=abs(my_dict[item]-gt_dict[item])
        if my_dict[item]==gt_dict[item]:
            tp+=1
        else:
            fp+=1
            
ARE=are_error/distinct
AAE=aae_error/distinct
print("Find:{}, TP:{}, FP:{}".format(len(gt_set & my_set),tp,fp))
print("Precision: {:8.4f}".format(precision))
print("ARE: {:8.6f}".format(ARE))
print("AAE: {:8.6f}".format(AAE))


EOF
['122', count: 72508, '8', count: 69349, '49', count: 59525, '124', count: 46414, '516', count: 46325, '232', count: 44572, '51', count: 39881, '171', count: 39569, '150', count: 39447, '121', count: 38702, '158', count: 35133, '146', count: 34204, '149', count: 34169, '878', count: 33879, '60', count: 33370, '308', count: 33035, '22', count: 30589, '379', count: 30370, '514', count: 29506, '1', count: 29156] 1536
Top-1536
Total memory 13008 bytes=Top-1536 with size 13008 bytes.
Execution time:7657.718 seconds.
Find:914, TP:180, FP:734
Precision:   0.5951
ARE: 0.193606
AAE: 1739.318359
